In [1]:
from spacy.tokenizer import Tokenizer
from spacy.lang.pl import Polish
from random import randint
from math import floor

In [2]:
def Levenstein(a: str, b: str):
    size_a: int = len(a)
    size_b: int = len(b)
    # prep
    L = [[0 for _ in range(size_b + 1)] for _ in range(size_a + 1)]
    for i in range(size_b + 1):
        L[0][i] = i

    for i in range(size_a + 1):
        L[i][0] = i

    for i in range(1, size_a + 1):
        for j in range(1, size_b + 1):
            cost: int = 0
            if a[i - 1] != b[j - 1]:
                cost = 1
            L[i][j] = min(L[i - 1][j] + 1, L[i][j - 1] + 1, L[i - 1][j - 1] + cost)
    return L

In [3]:
def find_differences(a: str, b: str):
    Lev = Levenstein(a, b)
    i: int = len(a)
    j: int = len(b)
    changes = []
    while i >= 0 and j >= 0:
        if i == 0 and j == 0:
            break
        if i >= 1 and j >= 1:
            if Lev[i - 1][j - 1] <= Lev[i - 1][j] and Lev[i - 1][j - 1] <= Lev[i][j - 1]:
                if a[i - 1] == b[j - 1]:
                    i, j = i - 1, j - 1
                    continue
                changes.append((i - 1, j - 1, 1))
                i, j = i - 1, j - 1
            elif Lev[i - 1][j - 1] > Lev[i][j - 1]:
                changes.append((i, j - 1, 2))
                j -= 1
            elif Lev[i - 1][j - 1] > Lev[i - 1][j]:
                changes.append((i - 1, j, 3))
                i -= 1
        elif i >= 1:
            changes.append((i - 1, j, 3))
            i -= 1
        elif j >= 1:
            changes.append((i, j - 1, 2))
            j -= 1

    changes.reverse()
    return changes

In [4]:
def visualize_differences(a: str, b: str):
    differences = find_differences(a, b)
    i: int = 0
    for operation in differences:
        shift: int = operation[0]
        char: str = b[operation[1]]
        if operation[2] == 1:
            print(a[:i + shift] + '*' + char + '*' + a[i + shift + 1:], "(zamiana litery", a[i + shift], "na", char,
                  ")")
            a = a[:i + shift] + char + a[i + shift + 1:]
        elif operation[2] == 2:
            print(a[:i + shift] + '+' + char + '+' + a[i + shift:], "(dodanie litery", char, ")")
            a = a[:i + shift] + char + a[i + shift:]
            i += 1
        else:
            print(a[:i + shift] + '_' + a[i + shift+1:], "(usunięcie litery", a[i+shift], ")")
            a = a[:i + shift] + a[i + shift+1:]
            i -= 1
    print(a)

## Wizualizacja przejść

In [5]:
pairs = [["los", "kloc"], ["Łódź", "Lodz"], ["kwintesencja", "quintessence"],
         ["ATGAATCTTACCGCCTCG", "ATGAGGCTCTGGCCCCTG"]]

for pair in pairs:
    print(pair[0], " ", pair[1])
    visualize_differences(pair[0], pair[1])
    print("@@@@@@@@")

los   kloc
+k+los (dodanie litery k )
klo*c* (zamiana litery s na c )
kloc
@@@@@@@@
Łódź   Lodz
*L*ódź (zamiana litery Ł na L )
L*o*dź (zamiana litery ó na o )
Lod*z* (zamiana litery ź na z )
Lodz
@@@@@@@@
kwintesencja   quintessence
*q*wintesencja (zamiana litery k na q )
q*u*intesencja (zamiana litery w na u )
quintes+s+encja (dodanie litery s )
quintessenc_a (usunięcie litery j )
quintessenc*e* (zamiana litery a na e )
quintessence
@@@@@@@@
ATGAATCTTACCGCCTCG   ATGAGGCTCTGGCCCCTG
ATGA*G*TCTTACCGCCTCG (zamiana litery A na G )
ATGAG*G*CTTACCGCCTCG (zamiana litery T na G )
ATGAGGCT+C+TACCGCCTCG (dodanie litery C )
ATGAGGCTCT*G*CCGCCTCG (zamiana litery A na G )
ATGAGGCTCTG*G*CGCCTCG (zamiana litery C na G )
ATGAGGCTCTGGC*C*CCTCG (zamiana litery G na C )
ATGAGGCTCTGGCCCCT_G (usunięcie litery C )
ATGAGGCTCTGGCCCCTG
@@@@@@@@


## LCS and differences


In [8]:
def tokenize(filename):
    res = []
    with open(filename, 'r', encoding="utf8") as file:
        T = file.readlines()
    tokenizer = Tokenizer(Polish().vocab)
    for t in T:
        chars = tokenizer(t)
        for c in chars:
            res.append(str(c))

    return res

def delete_tokens(text):
    new_text: list = text.copy()
    length: int = len(text)
    tokens_to_delete: int = floor(length * 0.03)
    for i in range(tokens_to_delete):
        p = randint(0, length - 1)
        new_text.pop(p)
        length -= 1
    return new_text

In [9]:
def LCS(a: list, b: list):
    size_a: int = len(a)
    size_b: int = len(b)
    C = [[0 for _ in range(size_b + 1)] for _ in range(size_a + 1)]
    commons = []
    for i in range(1, size_a + 1):
        for j in range(1, size_b + 1):
            if a[i - 1] == b[j - 1]:
                C[i][j] = C[i - 1][j - 1] + 1
            else:
                C[i][j] = max(C[i - 1][j], C[i][j - 1])
    i: int = len(a)
    j: int = len(b)
    while i > 0 and j > 0:
        if a[i - 1] == b[j - 1]:
            commons.append(a[i - 1])
            i -= 1
            j -= 1
        elif C[i - 1][j] > C[i][j - 1]:
            i -= 1
        else:
            j -= 1
    commons.reverse()
    return C, commons

In [10]:
def diff_tool(text, common_arr):
    text_index: int = 0
    arr_index: int = 0
    line: int = 1
    diffs = []
    while text_index < len(text) and arr_index < len(common_arr):
        if text[text_index] == common_arr[arr_index]:
            if text[text_index] == "\n":
                line += 1
            text_index += 1
            arr_index += 1
            continue
        if text[text_index] == "\n":
            text_index += 1
            line += 1
            continue
        elif text[text_index] != common_arr[arr_index]:
            diffs.append((text[text_index], line))
            text_index += 1
    return diffs

In [13]:
def LCS_from_tokens_and_diffs(chars):
    text_a: list = delete_tokens(chars)
    text_b: list = delete_tokens(chars)
    lcs, commons = LCS(text_a, text_b)
    # print(commons)
    print("Długość najdłuższego wspólnego podciągu tokenów:", lcs[len(text_a)][len(text_b)])
    print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
    print("Operacja diff dla zbioru tokenów:")
    diff_in_a = diff_tool(text_a, commons)
    diff_in_b = diff_tool(text_b, commons)
    for elem in diff_in_a:
        print(elem[0], "<", elem[1])
    for elem in diff_in_b:
        print(elem[0], ">", elem[1])

In [14]:
tokens = tokenize("romeo-i-julia-700.txt")

LCS_from_tokens_and_diffs(tokens)


Długość najdłuższego wspólnego podciągu tokenów: 2472
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Operacja diff dla zbioru tokenów:
książę < 10
  < 23
— < 30
domów, < 32
maski, < 32
treścią < 57
koty, < 86
Rozruchać < 109
z < 109
walecznym < 109
mur < 114
to < 124
zawsze < 124
mi < 134
sprawię. < 134
każdą < 144
do < 144
się < 149
Montekiego. < 149
z < 170
przyczyny! < 170
sobą, < 175
ścierpią. < 183
GRZEGORZ < 246
TYBALT < 288
jak < 291
wchodzą < 295
MONTEKI < 334
spokój < 354
synowcze, < 375
zbiegł < 389
wschodu < 401
sykomorowy < 403
Tam, < 405
na < 422
zajdzie, < 429
z < 440
/ < 472
tam, < 523
Taką < 546
moja < 546
                         < 562
sprawił, < 585
                         < 602
jęczyć < 603
Nie < 608
do < 609
ROMEO < 626
Grotów < 640
- < 662
przestać < 673
myśleć. < 673
rodu, > 11
nieprzyjaznych > 12
* > 14
małżonka > 28
Jan > 43
nieszczęśliwych > 52
jak > 56
Wchodzą > 71
i > 71
Ma > 81
rozumieć, > 81
tego > 114
trzymają. > 118
Pobiwszy > 132
GRZEGORZ > 165
nas, > 186
skrzywiłem >